In [43]:
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler

elec = pd.read_csv('..\dataset generation\small_elec_features.csv')
elec['Class'] = 'elec'
elec = elec.iloc[2:, 1:]
#print(elec.head())
#print(elec.shape)

rock = pd.read_csv('..\dataset generation\small_rock_features.csv')
rock['Class'] = 'rock'
rock = rock.iloc[2:, 1:]
#print(rock.head())
#print(rock.shape)

df = pd.concat([elec, rock])
df = df.sample(frac=1).reset_index(drop=True)
print(df.head())
print(df.shape)

y_data = df['Class']
x_data = df.drop('Class', axis = 1)

class_data = pd.get_dummies(df['Class'], drop_first = False)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(x_data)
x_df = DataFrame(scaled)

df = pd.concat([x_df, class_data], axis = 1)
df = df.dropna()
print(df.head())
print(df.shape)

           chroma_cens        chroma_cens.1         chroma_cens.2  \
0  0.14048266410827637  0.35956239700317383   -0.8183038234710693   
1   -0.531083345413208  -0.9379549026489258   -1.0180222988128662   
2  -1.0480496883392334  -0.7976562976837158     1.718958854675293   
3  -1.2076154947280884   0.9171900749206543   -1.0187931060791016   
4  -0.8375504016876221   -1.179531455039978  -0.12715697288513184   

         chroma_cens.3        chroma_cens.4         chroma_cens.5  \
0   -1.027938723564148  -1.0635091066360474   -0.8256161212921143   
1  -0.1936349868774414   1.1118130683898926  -0.47368359565734863   
2  0.05180811882019043    1.497988224029541    0.9983131885528564   
3  -0.4397881031036377  -0.7992105484008789         1.55126953125   
4  -0.7527523040771484  -0.7323260307312012   -0.1390528678894043   

         chroma_cens.6         chroma_cens.7        chroma_cens.8  \
0  -1.2434890270233154   -0.4564228057861328  -0.6003592014312744   
1  -0.7546899318695068   -0.2933

In [44]:
from sklearn.model_selection import train_test_split

y_cols = ['rock', 'elec']
y_data = df[y_cols]
x_data = df.drop(y_data, axis = 1)

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = 0.1)

In [45]:
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 300, hidden_layer_sizes = (200,100), max_iter = 500)
mlp.fit(x_training_data, y_training_data)
predictions = mlp.predict(x_test_data)

print("Confusion Matrix:")
print(multilabel_confusion_matrix(y_test_data.values.argmax(axis=1), predictions.argmax(axis=1)))
print("\nAccuracy:", accuracy_score(y_test_data, predictions))
print("\nMSE:", mean_squared_error(y_test_data, predictions))
print("\nClassification Report:")
print(classification_report(y_test_data, predictions))

Confusion Matrix:
[[[86  8]
  [11 95]]

 [[95 11]
  [ 8 86]]]

Accuracy: 0.905

MSE: 0.095

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91       106
           1       0.89      0.91      0.90        94

   micro avg       0.91      0.91      0.91       200
   macro avg       0.90      0.91      0.90       200
weighted avg       0.91      0.91      0.91       200
 samples avg       0.91      0.91      0.91       200

